# 심층 신경망

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/7-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

인공 신경망은 층을 많이 추가할 수 있다. 그래서 딥 러닝.

## 2개의 층

In [6]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [7]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0 # 표준화 스케일링  ->  픽셀값을 0~1 사이로
train_scaled = train_scaled.reshape(-1, 28*28) # 784 크기의 픽셀을 1차원으로

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [8]:
# 1절을 만든 신경망 모델과 다른 점은 입력층과 출력층 사이에 밀집층이 추가된 것.
# 이 추가된 밀집층을 은닉층
# 은닉층에는 주황색 원으로 활성화 함수가 표시되어 있다.
# 은닉층에서 선형적인 산술계산만 수행한다면 수행 역할이 없는 셈.
# 그래서 선형을 비 선형으로 비틀어줘야 함.
# 은닉층은 층 사이에서 산술 계산을 도와주고, 출력층에 대한 확률을 예측하는 데 정보를 준다.

dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
# 케라스의 첫번쨰 층은 반드시 매개변수 input_shape를 넣어주어야 한다.
# dense1이 은닉층이고, 100개의 뉴런을 가진 밀집층. 뉴런 갯수를 정하는 데는 특별한기준x
dense2 = keras.layers.Dense(10, activation='softmax')
# 출력층

## 심층 신경망 만들기

In [9]:
model = keras.Sequential([dense1, dense2]) # 리스트로 넣어주되, 출력층을 가장 마지막에
# 인공신경망의 강력한 성능은 이렇게 층을 추가하여 입력 데이터에 대해 연속적인 학습을
# 진행하는 능력에서 나온다.
# 앞에서 배운 선형 회귀, 로지스틱 회귀, 결정 트리 등 다른 머신러닝 알고리즘들과는 대조.


In [10]:
model.summary() # 층에 대한 유용한 정보

# 출력크기는 왜 None? 샘플 갯수가 아직 정해져있ㅎ지 않아서
# 케라스 모델의 fit메서드에 훈련 데이터를 주입하면 데이터들을 한번에 사용하지 않고
# 나누어 여러번에 걸쳐 경사하강법 단계를 수행하기 떄문., = 미니 배치 경사하강법
# 어떤 batch size로도 유연하게 대응할 수 있도록 none으로 결정.

# 100은 뉴런의 갯수고, 100개의 출력이 나올 것.
# 샘플마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축되었다.
# 마지막으로 모델파라미터 개수 : 입력 픽셀 784개 x 100개의 모든 조합에 대한 가중치
    # 입력층에 784개의 뉴런 , 출력층에 100개의 뉴런.  + 뉴런마다 1개의 절편도 있다.
        # = 78500
        
    # 출력층에서는 은닉층을 통과한 100개로 압축된 특성이자 뉴런과 10개의 뉴런의 조합
        # 으로 모두 연결되고(+활성화 함수 존재.) + 뉴런마다 1개 절편
            # 1000+10 = 1010 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 1ms/step - loss: 0.5666 - accuracy: 0.8066
Epoch 2/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.4108 - accuracy: 0.8517
Epoch 3/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3746 - accuracy: 0.8649
Epoch 4/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3521 - accuracy: 0.8730
Epoch 5/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3348 - accuracy: 0.8791


## 층을 추가하는 다른 방법

In [12]:
# 시퀀셜 클래스에 층을 추가하는 방법.
model = keras.Sequential([ # 층을 한 눈에 쉽게 알아볼 수 있는 방법
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output') # 층마다 name을 붙여줄 수 있다.
], name='패션 MNIST 모델') # 모델도 이름 붙여줄 수 있네 다른 것과 구별하기 위해서

In [13]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

# 얘는 왜 안되지?

Epoch 1/5


ValueError: in user code:

    File "C:\Users\Playdata\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Playdata\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Playdata\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Playdata\anaconda3\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Playdata\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Playdata\anaconda3\lib\contextlib.py", line 492, in enter_context
        result = _cm_type.__enter__(cm)
    File "C:\Users\Playdata\anacon

In [16]:
model = keras.Sequential()
# 또 층을 추가하는 방법
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)
# 추가된 층이 성능을 향상시켰다는 걸 알 수 있다.

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5623 - accuracy: 0.8081
Epoch 2/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.4084 - accuracy: 0.8525
Epoch 3/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3744 - accuracy: 0.8652
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3518 - accuracy: 0.8725
Epoch 5/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3353 - accuracy: 0.8786


## 렐루 활성화 함수
: 초창기 인공 신경망의 은닉층에 많이 사용된 함수는 시그모이드.
하지만 함수의 오른쪽, 왼쪽 끝으로 갈수록 그래프가 누워있어서 올바른 출력을 만드는데
신속하게 대응X
특히 층이 많은 심층신경망일수록 그 효과가 누적되어 학습 더어려워짐.

이를 개선하기위한 활성화 함수가 렐루
렐루는 입력이 양수일 경우 마치 활성화 함수가 없는거서첢 그냥 입력을 통과.
음수일 경우에는 0

렐루함수는 max(0,z)와 같이 쓸 수 있고, z가 0보다 크면 z를 출력, z가 0보다 작으면 0을 출력한다.
렐루는 특히 이미지 처리에서 좋은 성능.

In [21]:
model = keras.Sequential() 
model.add(keras.layers.Flatten(input_shape=(28, 28))) # 2828크기라서 인공신경망에
# 주입하기 위해 넘파이 배열의 reshape메서드를 사용해 1차원으로 펼쳐왓었는데
#케라스에서는 플래튼 층을 제공한다.
# flatten 클래스는 배치차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할만 한다.
# 입력에 곱해지는 가중치나 절편이 없다. 따라서 인공 신경망의 성능을 위해
# 기여하는 바는 없다., 입력층과 은닉층 사이에 있으니 이를층이라 부른다.

# 플래툰 클래스는 학습하는 층도 아니야.
# 모델 파라미터 개수 : 0 

# 케라스의 플래툰 층을 신경망 모델에 추가하면 입력값의 차원을 짐작할 수 있다는 것이 
# 또 하나의 장점 : 앞의 출력에서 784개의 입력이 첫번째 은닉층에 전달된다는 것을 알 수 있다.
# 이는 이전에 만들었던 모델에서는 쉽게 눈치채기 어려웠던 것.

# 입력 데이터에 대한 전처리 과정을 가능한 모델에 포함시키는 것이 케라스 API의 철학 중 하나.
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [22]:
(train_input, train_target), (test_input, test_target) =\
keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [23]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
# 확실히 시그모이드 함수보다 relu함수가 성능이 좋다.
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5309 - accuracy: 0.8138
Epoch 2/5
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3954 - accuracy: 0.8588
Epoch 3/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3580 - accuracy: 0.8715
Epoch 4/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3354 - accuracy: 0.8801
Epoch 5/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3210 - accuracy: 0.8861


In [24]:
model.evaluate(val_scaled, val_target) 
# 검증 세트에서의 성능은 1절의 은닉층을 추가하지 않은 경우보다 몇퍼센트 성능이 향상.

375/375 [==============================] - 1s 1ms/step - loss: 0.3576 - accuracy: 0.8792


[0.3576281666755676, 0.8792499899864197]

## 옵티마이저
: 하이퍼 파라미터는 신경망에 특히 많다.
은닉층의 개수, 뉴런 개수, 활성화 함수, 층의 종류, 배치 사이즈 매개변수, 에포크 매개변수 등

추가할 은닉층의 개수는 모델이 학습하는 것이 아니라 우리가 지정해주어야 할 하이퍼파라미터.

은닉층의 뉴런 개수 조차 하이퍼 파라미터.

이 장에서는 가장 기본적인 밀집층만 다루지만, 다른 종류의 층을 선택할 수도 있다.

케라스는 기본적으로 미니배치 경사 하강법을 사용하여 미니배치 개수 32개다.
fit()메서드의 batch_size 매개변수에서 이를 조정할 수 있으며 이 역시 하이퍼파라미터.


조정해야 할 하이퍼 파라미터가 정말 많다.
처음부터 모델을 구성하고, 각종 하이퍼 파라미터 최적값을 찾는 것은 어려운 작업.
여러가지 옵티마이저를 테스트.


마지막 compile 메소드에서는 케라스의 기본 경사 하강법 알고리즘인
RMSprop를 사용할건데, 케라스는 '다양한 종류의 경사 하강법 알고리즘'을 제공.
이들을 옵티마이저 라고 한다.
 
RMSprop의 학습률 또한 조정할 하이퍼 파라미터 중 하나.

가장 기본적인 옵티마이저는 확률적 경사 하강법인 SGD다.
이름이 SGD이지만 1개의 샘플을 뽑아서 훈련하지 않고 앞서 언급한 것처럼 기본적으로
미니 배치를 사용

SGD 옵티마이저를 사용하려면 complie 메서드의 optimizer 매개변수를 sgd로 저장.

이 옵티마이저는 tensorflow.keras.optimizer 패키지 아래 sgd 클래스로 구현되어 있다.


In [26]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy'
              , metrics='accuracy')
# sgd 문자열은 이 클래스의 기본 설정 매개변수로 생성한 객체와 동일

In [27]:
sgd = keras.optimizers.SGD()
# sgd 문자열은 이 클래스의 기본 설정 매개변수로 생성한 객체와 동일.
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy'
              , metrics='accuracy')

# 즉 바로 위의 코드와 100% 일치.

In [28]:
sgd = keras.optimizers.SGD(learning_rate=0.1)
# 만약 SGD 클래스의 학습률 기본값이 0.01일 때 이를 바꾸고 싶다면 다음과 같이
# 원하는 학습률을 learning_rate 매개변수에 지정하여 사용.

In [29]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)
# 기본 경사 하강법 옵티마이저는 모두 SGD 클래스에서 제공.
# SGD 클래스의 모멘텀 매개변수의 기본값 = 0
# 이를 0보다 큰 값으로 지정하면 마치 이전의 그레이디언트를 가속도처럼 사용하는
# 모멘텀 최적화를 사용합니다. 보통 모멘텀 매개변수는 0.9이상을 지정.

# 모멘텀을 대신하여 SGD를 사용하여 신경망 학습 가능.
# 그럼에도 불구하고 모멘텀 사용 이유 : 손실된 값들이 발생하거나
# 로컬 최솟값에 빠지는 부분 방지할 수 있기 때문.
# 모멘텀은 이러한 다소 극단적인 그래프 결과가 산출되지 않도록 부드럽게 결과를 산출해주는 역할을 수행

# SGD 클래스의 nesterov 매개변수를 기본값 False에서 True로 바꾸면
# 네스테로프 모멘텀 최적화를 사용한다.
# 네스테로프 모멘텀은 모멘텀 최적화를 2번 반복 구현. 대부분의 경우 네모 최적화가
# 기본 확률적 경사 하강법보다 더 나은 성능.

# 학습률 : 머신러닝에서 학습되는 양 또는 단계.
# 모델이 최적점에 가까이 갈수록 학습률을 낮출 수 있다. => 안정적으로 최적점에 수렴할 가능성
# 이런 학습률을 적응적 학습률 - 이런 방식들은 학습률 매개변수를 튜닝하는 수고를 덜 수 O


In [30]:
# 적응적 학습률을 사용하는 대표적 옵티는 Adagrad와 RMSprop이다.
# 각각 compile 메서드의 옵티마이저 메개변수에 adagrad, RMSprop으로 지정할 수 있다.
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy'
              , metrics='accuracy')

In [ ]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy'
              , metrics='accuracy')

In [30]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [20]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy'
              , metrics='accuracy')

# 모멘텀 최적화와 RMSprop의 장점을 접목한 것이 Adam이다.
# Adam은 RMSprop과 함께 맨 처음 시도해 볼 수 있는 좋은 알고리즘.
# 적응적 학습률을 사용하는 3개의 클래스는 학습률 매개변수의 기본값으로 모두 0.001을 사용.

# compile()메서드의 옵티마이저를 adam으로 설정하고 5번 에포크 훈련
# 출력 결과를 보면 RMSprop 사용햇을 때와 거의 같은 성능.
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.5266 - accuracy: 0.8148
Epoch 2/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3919 - accuracy: 0.8585
Epoch 3/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3495 - accuracy: 0.8730
Epoch 4/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3236 - accuracy: 0.8813
Epoch 5/5
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3046 - accuracy: 0.8884


In [28]:
model.evaluate(val_scaled, val_target)
# 검증세트에서는 기본 RMSprop보다 조금 나은 성능 보여줌.

375/375 [==============================] - 0s 1ms/step - loss: 0.3544 - accuracy: 0.8730


[0.35439205169677734, 0.8730000257492065]

이번 장에서 여러 개의 층을 추가하여 다층 인공 신경망(심층신경망)을 만드는 방법 터득.
또 케라스 API를 사용하여 층을 추가하는 여러가지 방법

케라스 모델의 정보를 요약해주는 summary() 메서드

출력값의 의미를 이해하고 모델 파라미터 개수를 계산해 맞추어도 보고,
모델 파라미터 개수를 계산하는 과정은 모델을 올바르게 이해하고 있는지 확인하는 좋은 방법

은닉층에 적용한 시그모이드 활성화 함수 대신에 새로운 렐루 활성화 함수에 대해 배웠고,
이를 적용해 약간의 성능향상.

또한 다양한 고급 경사 하강법 옵티들을 적용하는 방법도 살펴봄.

케라스API를 사용하면 이런 작업이 어렵지 않고 직관적으로 구성할 수 있다.